In [ ]:
!pip install --upgrade torch
!pip install --upgrade transformers
!pip install --upgrade torchvision
!pip install langdetect
!pip install -U accelerate
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c 'petfinder-adoption-prediction'

Saving kaggle.json to kaggle.json
rm: cannot remove '/root/.kaggle': No such file or directory
 99% 1.92G/1.94G [00:08<00:00, 258MB/s]
100% 1.94G/1.94G [00:08<00:00, 241MB/s]


In [ ]:
from langdetect import detect
import pandas as pd
import zipfile
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import os
import json
import time
from queue import Queue
from threading import Thread, Lock
from tqdm import tqdm
from google.colab import drive
import signal

In [ ]:
# Unzip the downloaded data
data_zip_path = "/content/petfinder-adoption-prediction.zip"
with zipfile.ZipFile(data_zip_path, 'r') as zip_ref:
  zip_ref.extractall("/content")  # Extract to the same directory

In [ ]:
# Load the training data (path adjusted after extraction)
data_path = "/content/train/train.csv"
df = pd.read_csv(data_path)

# Set Pandas option to display all columns
pd.set_option('display.max_colwidth', None)

test_data_path = "/content/test/test.csv"
test_df = pd.read_csv(test_data_path)


In [ ]:
# Check GPU availability
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Load translation model
translation_model_name = "Helsinki-NLP/opus-mt-mul-en"
translation_tokenizer = AutoTokenizer.from_pretrained(translation_model_name)
translation_model = AutoModelForSeq2SeqLM.from_pretrained(translation_model_name)
translator = pipeline('translation', model=translation_model, tokenizer=translation_tokenizer, device=device)

# Load LLaMA model
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=device)


# Define the features to extract
features = [
    "Is the pet described as friendly?",
    "Does the pet require special care?",
    "Is the pet currently with the owner?",
    "Is the pet good with other animals?",
    "Is the pet's adoption urgent?",
    "Does the pet need a lot of space?",
    "Is the pet described as lovely or loveable?",
    "Is there a contact person mentioned?",
    "Must these pets be adopted as a group?",
    "Does the pet need medical care?",
    "must always be kept indoors"
]

Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/707k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:

  # Function to set language column in DataFrame
def set_language(df):
    def detect_language(text):
      try:
          lang = detect(text)
          return lang
      except Exception as e:
          return 'unknown'

    # Create a new column 'Language' to store detected language
    df['Language'] = df['Description'].apply(detect_language)
    return df

def translate_to_english(df, translate_column="Description", translated_column="Translated_Description"):

    # Filter rows where Language is not 'en'
    non_english_texts = df[~df['Language'].isin(['en', 'unknown'])]

    print("rows to translate: ", len(non_english_texts))
    # Translate non-English texts
    texts_to_translate = []
    for text_t_t in non_english_texts[translate_column].tolist():
      texts_to_translate.append(text_t_t[0:460])

    translations = translator(texts_to_translate)

    # Map translations back to DataFrame
    translated_texts = [translation["translation_text"] for translation in translations]
    non_english_texts[translated_column] = translated_texts
    # Merge translated results back into the original DataFrame
    
    # este merge parece estar mal
    df = pd.merge(df, non_english_texts[[translate_column, translated_column]], on=translate_column, how='left')

    # if Translated_Description is NaN replace it by Description
    df[translated_column] = df.apply(lambda row: row[translate_column] if pd.isna(row[translated_column]) and row['Language'] == 'en' else row[translated_column], axis=1)


    return df

# Function to create new features using the classifier pipeline
def create_new_features(descriptions):
    feature_predictions = classifier(descriptions, candidate_labels=features, multi_label=True)
    return feature_predictions

In [ ]:
drive.mount('/content/drive')
# Ensure the processed data directory exists in Google Drive
processed_data_dir = '/content/drive/MyDrive/ProcessedData'
if not os.path.exists(processed_data_dir):
    os.makedirs(processed_data_dir)

Mounted at /content/drive


In [ ]:
df = set_language(df)
df = translate_to_english(df)
df.loc[df['Language'] == 'unknown', 'Description'] = 'No info'
df.loc[df['Language'] == 'unknown', 'Translated_Description'] = 'No info'
checkpoint = "/content/drive/MyDrive/ProcessedData/df_checkpoint_1.csv"
df.to_csv(checkpoint, index=False)

rows to translate:  844


<ipython-input-7-8ef24da9d495>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_english_texts[translated_column] = translated_texts


In [ ]:
test_df = set_language(test_df)
test_df = translate_to_english(test_df)
test_df.loc[df['Language'] == 'unknown', 'Description'] = 'No info'
test_df.loc[df['Language'] == 'unknown', 'Translated_Description'] = 'No info'
checkpoint = "/content/drive/MyDrive/ProcessedData/test_df_checkpoint_1.csv"
test_df.to_csv(checkpoint, index=False)

rows to translate:  227


<ipython-input-7-8ef24da9d495>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_english_texts[translated_column] = translated_texts


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ProcessedData/df_checkpoint_1.csv")
test_df = pd.read_csv("/content/drive/MyDrive/ProcessedData/test_df_checkpoint_1.csv")

In [ ]:
new_features = create_new_features(df['Translated_Description'].tolist())
if new_features is not None:
    for feature in features:
        df[feature] = [prediction['scores'][prediction['labels'].index(feature)] if feature in prediction['labels'] else None for prediction in new_features]
checkpoint = "/content/drive/MyDrive/ProcessedData/df_checkpoint_2.csv"
df.to_csv(checkpoint, index=False)

In [ ]:
# replace nan with No info
test_df['Translated_Description'] = test_df['Translated_Description'].fillna('No info')

In [ ]:
new_features = create_new_features(test_df['Translated_Description'].tolist())
if new_features is not None:
    for feature in features:
        test_df[feature] = [prediction['scores'][prediction['labels'].index(feature)] if feature in prediction['labels'] else None for prediction in new_features]
checkpoint = "/content/drive/MyDrive/ProcessedData/test_df_checkpoint_2.csv"
test_df.to_csv(checkpoint, index=False)

KeyboardInterrupt: 